# Environment

In [1]:
using Pkg; Pkg.activate("closures-in-action")
Pkg.add("HTTP")
Pkg.add("Sockets")

  Activating project at `~/dev/nbs/closures-in-action`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/dev/nbs/closures-in-action/Project.toml`
  No Changes to `~/dev/nbs/closures-in-action/Manifest.toml`
   Resolving package versions...
  No Changes to `~/dev/nbs/closures-in-action/Project.toml`
  No Changes to `~/dev/nbs/closures-in-action/Manifest.toml`


In [2]:
using HTTP, Sockets

## Quick memory jogger
The following links could help if you are unfamiliar with these concepts:
- https://medium.com/r/?url=https%3A%2F%2Ftowardsdatascience.com%2Fwhat-on-earth-are-closures-a4f9c7c652d2
- https://medium.com/r/?url=https%3A%2F%2Ftowardsdatascience.com%2Fwhat-on-earth-is-an-anonymous-function-f8043eb845f3

In [3]:
n = function hello(x, y)
   println("hi") 
end

hello (generic function with 1 method)

In [4]:
n(5, 1)

hi


# Closures In Action

In [5]:
mutable struct Route
    path::String
    page::Any
    function Route(path::String = "", page::Any = "")
        new(path, page)
    end
end

In [6]:
function funcdefs(routes::AbstractVector, ip::String, port::Integer)
    add(r::Route) = push!(routes, r)
    remove(i::Int64) = deleteat!(routes, i)
    start() = _start(routes, ip, port)
    return(add, remove, start)
end

funcdefs (generic function with 1 method)

In [7]:
function _start(routes::AbstractVector, ip::String, port::Integer)
    server = Sockets.listen(Sockets.InetAddr(parse(IPAddr, ip), port))
    println("Starting server on port ", string(port))
    routefunc = generate_router(routes, server)
    @async HTTP.listen(routefunc, ip, port; server = server)
    println("Successfully started server on port ", port, "\n")
    println("You may visit it now at http://" * string(ip) * ":" * string(port))
    return(server)
end
function generate_router(routes::AbstractVector, server)
    route_paths = Dict([route.path => route.page for route in routes])
    routeserver = function serve(http)
     HTTP.setheader(http, "Content-Type" => "text/html")
        fullpath = http.message.target
    if contains(http.message.target, '?')
         fullpath = split(http.message.target, '?')
         args = ""
    end
     if length(fullpath) > 1
         args = fullpath[2]
     end
     if fullpath in keys(route_paths)
        write(http, route_paths[fullpath])
     else
         write(http, route_paths["404"])
     end
 end # serve()
    return(routeserver)
end
function stop!(x::Any)
    close(x)
end

stop! (generic function with 1 method)

In [8]:
mutable struct OurServer
    ip::String
    port::Integer
    routes::AbstractVector
    remove::Function
    add::Function
    start::Function
    function OurServer(ip::String, port::Int64)
        routes = []
        add, remove, start = funcdefs(routes, ip, port)
        new(ip, port, routes, remove, add, start)
    end

    function OurServer()
        port = 8001
        ip = "127.0.0.1"
        OurServer(ip, port)
    end
end

In [9]:
home = Route("/", "<h1>HELLO WORLD</h1>")

Route("/", "<h1>HELLO WORLD</h1>")

In [10]:
four04 = Route("404", "<h1> Directory not found </h1>")

Route("404", "<h1> Directory not found </h1>")

In [11]:
server = OurServer()

OurServer("127.0.0.1", 8001, Any[], var"#remove#2"{Vector{Any}}(Any[]), var"#add#1"{Vector{Any}}(Any[]), var"#start#3"{Vector{Any}, String, Int64}(Any[], "127.0.0.1", 8001))

In [12]:
server.add(home)

1-element Vector{Any}:
 Route("/", "<h1>HELLO WORLD</h1>")

In [13]:
server.add(four04)

2-element Vector{Any}:
 Route("/", "<h1>HELLO WORLD</h1>")
 Route("404", "<h1> Directory not found </h1>")

In [19]:
serving = server.start()

Starting server on port 8001
Successfully started server on port 8001

You may visit it now at http://127.0.0.1:8001


Sockets.TCPServer(RawFD(45) active)

In [20]:
stop!(serving)

# Anonymous Functions

In [21]:
x = [5, 10, 15, 20, 25]

5-element Vector{Int64}:
  5
 10
 15
 20
 25

In [23]:
x -> x > 15

#11 (generic function with 1 method)

In [24]:
filter!(x -> x > 15, x)

2-element Vector{Int64}:
 20
 25

# Begin/end syntax

In [31]:
function examp(f::Function, x::String)
    f(x)
end

examp (generic function with 1 method)

In [32]:
examp("hello") do
    println("Hello")
end

LoadError: MethodError: no method matching (::var"#19#20")(::String)
[0mClosest candidates are:
[0m  (::var"#19#20")() at In[32]:2